<a href="https://colab.research.google.com/github/njafarov/Optimization_MILP/blob/main/Challenge_6_Network_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyomo
!apt-get install -y -qq glpk-utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.7 MB 5.1 MB/s 
     |████████████████████████████████| 49 kB 2.9 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123942 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Pr

In [2]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [8]:
# Define the model
model = pyo.ConcreteModel()

# Define sets
model.i = pyo.RangeSet(1,5)
model.j = pyo.RangeSet(1,9)
model.p = pyo.RangeSet(1,2)

# Define parameters
model.VarCost = pyo.Param(model.i, initialize={1:8,2:10,3:25,4:30,5:50})
VarCost = model.VarCost # Variable cost of WH 

model.FixCost = pyo.Param(model.i, initialize={1:100000,2:90000,3:80000,4:70000,5:60000})
FixCost = model.FixCost # Fixed cost of WH

model.InCost = pyo.Param(model.i,model.p, initialize={(1,1):77,(1,2):54,
                                                      (2,1):48,(2,2):33,
                                                      (3,1):41,(3,2):21,
                                                      (4,1):57,(4,2):94,
                                                      (5,1):94,(5,2):44})
InCost = model.InCost #Cost of transportation from plants to Cnetral Warehouse

model.P_cost = pyo.Param(model.p,initialize={1:8,2:13})
P_cost = model.P_cost #Production cost

model.Cap = pyo.Param(model.p, initialize={1:1500,2:500})
Cap = model.Cap # Capacity of Plants


model.D = pyo.Param(model.j, initialize={1:140,2:180,3:240,4:210,5:175,6:130,7:320,8:280,9:160})
D = model.D # Demand of regional warehouse

model.L = pyo.Param(model.i, model.j, initialize={(1,1):89,(1,2):78,(1,3):69,(1,4):95,(1,5):85,(1,6):73,(1,7):88,(1,8):73,(1,9):86,
                                                  (2,1):90,(2,2):91,(2,3):63,(2,4):61,(2,5):94,(2,6):58,(2,7):89,(2,8):90,(2,9):55,
                                                  (3,1):81,(3,2):82,(3,3):71,(3,4):98,(3,5):62,(3,6):71,(3,7):85,(3,8):75,(3,9):72,
                                                  (4,1):73,(4,2):97,(4,3):55,(4,4):75,(4,5):86,(4,6):54,(4,7):62,(4,8):99,(4,9):59,
                                                  (5,1):85,(5,2):98,(5,3):93,(5,4):82,(5,5):87,(5,6):58,(5,7):98,(5,8):72,(5,9):50})
L = model.L # Cost per distance from Central warehouse to regional warehouse

# Decision Variable
model.x = pyo.Var(model.i, model.j, within=pyo.NonNegativeReals)
x = model.x
model.z = pyo.Var(model.i, model.p, within=pyo.NonNegativeReals)
z = model.z
model.y = pyo.Var(model.i, within=pyo.Binary)
y=model.y

def objective_function(model):
  return sum(sum(x[i,j]*L[i,j] for i in model.i)for j in model.j)+sum(FixCost[i]*y[i] for i in model.i)+sum(sum(z[i,p]*InCost[i,p] for i in model.i) for p in model.p)+sum(sum(z[i,p] for i in model.i)*P_cost[p] for p in model.p)+sum(sum(x[i,j]*VarCost[i] for i in model.i)for j in model.j)
  

model.Obj = pyo.Objective(rule = objective_function, sense=pyo.minimize)


def cons1(model,p):
  return sum(z[i,p] for i in model.i)<=Cap[p] 
model.Cons1 = pyo.Constraint(model.p,rule=cons1)


def cons2(model,i):
  return sum(z[i,p] for p in model.p) - sum(x[i,j] for j in model.j)==0
model.Cons2 = pyo.Constraint(model.i, rule=cons2)


def cons3(model,i):
  return sum(x[i,j] for j in model.j) -y[i]*1835<=0
model.Cons3 = pyo.Constraint(model.i, rule=cons3)


def cons4(model,j):
  return sum(x[i,j] for i in model.i)>=D[j]
model.Cons4 = pyo.Constraint(model.j, rule=cons4)


def cons5(model,i):
  return sum(x[i,j] for j in model.j)<=1835
model.Cons5 = pyo.Constraint(model.i, rule=cons5)


def cons6(model,i):
  return sum(y[i] for i in model.i)==1
model.Cons6 = pyo.Constraint(model.i,rule=cons6)


# Optimization
Solver = SolverFactory('glpk')
results = Solver.solve(model)

print("Objective funct: ",model.Obj())

for i in model.i:
  for p in model.p:
    print("From plant ",p,z[i,p](),"amount of materials goes to warehouse",i)

for i in model.i:
  for j in model.j:
    print("from Warehouse ",i, x[i,j](),"amount goes to Warehouse ",j)



Objective funct:  349490.0
From plant  1 0.0 amount of materials goes to warehouse 1
From plant  2 0.0 amount of materials goes to warehouse 1
From plant  1 1335.0 amount of materials goes to warehouse 2
From plant  2 500.0 amount of materials goes to warehouse 2
From plant  1 0.0 amount of materials goes to warehouse 3
From plant  2 0.0 amount of materials goes to warehouse 3
From plant  1 0.0 amount of materials goes to warehouse 4
From plant  2 0.0 amount of materials goes to warehouse 4
From plant  1 0.0 amount of materials goes to warehouse 5
From plant  2 0.0 amount of materials goes to warehouse 5
from Warehouse  1 0.0 amount goes to Warehouse  1
from Warehouse  1 0.0 amount goes to Warehouse  2
from Warehouse  1 0.0 amount goes to Warehouse  3
from Warehouse  1 0.0 amount goes to Warehouse  4
from Warehouse  1 0.0 amount goes to Warehouse  5
from Warehouse  1 0.0 amount goes to Warehouse  6
from Warehouse  1 0.0 amount goes to Warehouse  7
from Warehouse  1 0.0 amount goes to W